In [4]:
import re
import operator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from datetime import datetime

In [92]:
def save_plot(who, qcri, full, g_step, name, filename):
    full_graph = {}
    for t in full:
        if t not in full_graph:
            full_graph[t] = 1
        else:
            full_graph[t] += 1
    full_graph = pd.DataFrame(full_graph.items(), columns=['date', 'mentions'])
    full_graph = full_graph.sort_values(by=['date'])

    who_graph = {}
    for t in who:
        if t not in who_graph:
            who_graph[t] = 1
        else:
            who_graph[t] += 1
    who_graph = pd.DataFrame(who_graph.items(), columns=['date', 'mentions'])
    who_graph = who_graph.sort_values(by=['date'])

    qcri_graph = {}
    for t in qcri:
        if t not in qcri_graph:
            qcri_graph[t] = 1
        else:
            qcri_graph[t] += 1
    qcri_graph = pd.DataFrame(qcri_graph.items(), columns=['date', 'mentions'])
    qcri_graph = qcri_graph.sort_values(by=['date'])

    fig = plt.figure(figsize=(20,10))
    plt.plot(full_graph['date'], full_graph['mentions'], '-', color='green', label="both sources")
    plt.plot(qcri_graph['date'], qcri_graph['mentions'], '-', color='red', label="QCRI data")
    plt.plot(who_graph['date'], who_graph['mentions'], '-', color='blue', label="WHO data")
    

    plt.xlabel('Dates')
    plt.ylabel('Number of mentions / tweets from organisation')
    
    plt.xticks(rotation=90)
    plt.tick_params(axis='x', which='major', labelsize=7.5)
    plt.tick_params(axis='y', which='major', labelsize=7.5)
    plt.yticks(np.arange(0, max(full_graph['mentions']), g_step))
    plt.title(name, bbox={'facecolor': '0.8', 'pad': 5})
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper center', borderaxespad=0.)
    fig.savefig(filename + '_timeseries.png')
    plt.close()
    plt.show()

def read_files(name):
    who = []
    qcri = []
    full = []
    f = pd.read_json('data/qcri/' + name +'_geo.json', lines=True)
    for line in f['created_at']:
        dtime = datetime.strftime(datetime.strptime(str(line),'%Y-%m-%d %H:%M:%S'), '%Y %m %d')
        qcri.append(dtime)
        full.append(dtime)
    
    try:
        f = pd.read_csv('data/out/' + name + '.csv', 'rb', delimiter = '\t')
        for line in f['created_at']:
            dtime = datetime.strftime(datetime.strptime(line,'%a %b %d %H:%M:%S +0000 %Y'), '%Y %m %d')
            who.append(dtime)
            full.append(dtime)
    except:
        who = []

    if len(full) >= 100 and len(full) < 200 :
        save_plot(who, qcri, full, 5, name, name)
    elif len(full) >= 200 and len(full) < 500 :
        save_plot(who, qcri, full, 10, name, name)
    elif len(full) >= 500 and len(full) < 1000 :
        save_plot(who, qcri, full, 30, name, name)
    elif len(full) >= 1000 and len(full) < 10000 :
        save_plot(who, qcri, full, 50, name, name)
    elif len(full) > 10000 :
        save_plot(who, qcri, full, 100, name, name)

In [93]:
read_files('brazil_gov')